# TSA Chapter 5: GARCH(1,1) Model

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch5/TSA_ch5_garch/TSA_ch5_garch.ipynb)

Fitting and interpreting GARCH(1,1) model on S&P 500 returns.

In [ ]:
!pip install numpy pandas matplotlib scipy yfinance arch statsmodels -q

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from scipy import stats
import yfinance as yf
from arch import arch_model

In [ ]:
import yfinance as yf

# Download S&P 500 data
sp = yf.download('^GSPC', start='2000-01-01', end='2025-12-31', progress=False)
if isinstance(sp.columns, pd.MultiIndex):
    sp.columns = sp.columns.get_level_values(0)
sp500_returns = (sp['Close'].pct_change() * 100).dropna()
sp500_returns = pd.Series(sp500_returns.values, index=sp500_returns.index, name='returns')
print(f"S&P 500: {len(sp500_returns)} observations")

In [ ]:
# Fit GARCH(1,1)
am = arch_model(sp500_returns, vol='Garch', p=1, q=1, mean='Constant', dist='normal')
res = am.fit(disp='off')
print(res.summary())

omega = res.params['omega']; alpha = res.params['alpha[1]']; beta = res.params['beta[1]']
persistence = alpha + beta
uncond_vol = np.sqrt(omega / (1 - persistence))
half_life = np.log(0.5) / np.log(persistence)
print(f"\nPersistence (alpha+beta): {persistence:.4f}")
print(f"Unconditional volatility: {uncond_vol:.4f}%")
print(f"Half-life of shocks: {half_life:.1f} days")